In [241]:
import pandas as pd
import numpy as np
import json
from bs4 import BeautifulSoup
import re
import plotly.graph_objs as go
import random

from sklearn.manifold import TSNE
from sklearn.metrics.pairwise import cosine_similarity

import pymorphy3
from nltk.corpus import stopwords
from gensim.utils import simple_preprocess
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from transformers import BertTokenizer, BertModel
import torch

import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

# Config

In [242]:
SEED = 2023
RANDOM_STATE = 2023
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)

# Experiment 3 (Doc2Vec + BERT)
NROWS = 30000
DOC2VEC_VECTOR_SIZE = 64
DOC2VEC_MIN_COUNT = 6
DOC2VEC_EPOCHS = 100
DOC2VEC_ALPHA = 0.001
DOC2VEC_MODEL_PATH = '../models/rjm_doc2vec_30000rows_exp3.model'
BERT_MODEL_NAME = "DeepPavlov/rubert-base-cased"
BERT_MAX_LENGTH = 512

# Data


In [243]:
def clean_html(text):
    if pd.isna(text):
        return np.nan
    soup = BeautifulSoup(text, "html.parser")
    cleaned_text = soup.get_text(separator=" ")
    cleaned_text = cleaned_text.replace("\r", " ").replace("\n", " ")
    cleaned_text = re.sub(r"\s+", " ", cleaned_text)
    return cleaned_text.strip()


def clean_divis(text):
    if pd.isna(text) or text.isdigit():
        return np.nan
    return text.replace("-", " ")


def map_digits(digits):
    if pd.isna(digits):
        return np.nan
    try:
        num = int(digits)
        if num < 0 or num > 1500000:
            return np.nan
        return num
    except ValueError:
        return np.nan


def map_boolean_to_yes_no(value):
    if pd.isna(value):
        return np.nan
    return "Да" if value else "Нет"


def map_accommodation_type(value):
    if pd.isna(value):
        return np.nan
    accommodation_types = {
        "ROOM": "Комната",
        "FLAT": "Квартира",
        "DORMITORY": "Общежитие",
        "HOUSE": "Дом",
    }
    return accommodation_types.get(value, value)


def map_company_business_size(value):
    if pd.isna(value):
        return np.nan
    business_sizes = {
        "SMALL": "Малый бизнес",
        "MICRO": "Микробизнес",
        "LARGE": "Крупный бизнес",
        "BIG": "Большой бизнес",
        "MIDDLE": "Средний бизнес",
    }
    return business_sizes.get(value, value)


def map_transport_compensation(value):
    if pd.isna(value):
        return np.nan
    compensation_types = {
        "PASSAGE_PAID": "Оплачивается проезд",
        "FUEL_PAID": "Оплачивается топливо",
        "AUTO": "Предоставляется автомобиль",
    }
    return compensation_types.get(value, value)


def map_country(text):
    if pd.isna(text):
        return np.nan
    try:
        json_data = json.loads(text)
        if isinstance(json_data, list) and "countryName" in json_data[0]:
            return json_data[0]["countryName"]
    except json.JSONDecodeError:
        return np.nan
    return np.nan


def map_languageKnowledgeResume(text):
    if pd.isna(text):
        return np.nan
    try:
        json_data = json.loads(text)
        if isinstance(json_data, list) and json_data:
            languages = []
            for lang in json_data:
                if "codeLanguage" in lang and "level" in lang:
                    languages.append(
                        "{}: {}".format(lang["codeLanguage"], lang["level"])
                    )
            if languages:
                return "; ".join(languages)
    except json.JSONDecodeError:
        return np.nan
    return np.nan


def map_hardSkillsResume(text):
    if pd.isna(text):
        return np.nan
    try:
        json_data = json.loads(text)
        if isinstance(json_data, list) and json_data:
            hard_skills = [skill["hardSkillName"] for skill in json_data]
            return "; ".join(hard_skills)
    except json.JSONDecodeError:
        return np.nan
    return np.nan


def map_softSkillsResume(text):
    if pd.isna(text):
        return np.nan
    try:
        json_data = json.loads(text)
        if isinstance(json_data, list) and json_data:
            soft_skills = [skill["softSkillName"] for skill in json_data]
            return "; ".join(soft_skills)
    except json.JSONDecodeError:
        return np.nan
    return np.nan


def map_hardSkillsJob(text):
    if pd.isna(text):
        return np.nan
    try:
        json_data = json.loads(text)
        if isinstance(json_data, list) and json_data:
            hard_skills = [skill["hard_skill_name"] for skill in json_data]
            return "; ".join(hard_skills)
    except json.JSONDecodeError:
        return np.nan
    return np.nan


def map_languageKnowledgeJob(text):
    if pd.isna(text):
        return np.nan
    try:
        json_data = json.loads(text)
        if isinstance(json_data, list) and json_data:
            languages = []
            for lang in json_data:
                if "code_language" in lang and "level" in lang:
                    languages.append(
                        "{}: {}".format(lang["code_language"], lang["level"])
                    )
            if languages:
                return "; ".join(languages)
    except json.JSONDecodeError:
        return np.nan
    return np.nan


def map_softSkillsJob(text):
    if pd.isna(text):
        return np.nan
    try:
        json_data = json.loads(text)
        if isinstance(json_data, list) and json_data:
            soft_skills = [skill["soft_skill_name"] for skill in json_data]
            return "; ".join(soft_skills)
    except json.JSONDecodeError:
        return np.nan
    return np.nan


def map_driveLicenses(text):
    if pd.isna(text):
        return np.nan
    try:
        json_data = json.loads(text)
        if isinstance(json_data, list) and json_data:
            return ", ".join(json_data)
    except json.JSONDecodeError:
        return np.nan
    return np.nan


def map_educationList(text):
    if pd.isna(text) or text == "[]":
        return np.nan
    try:
        json_data = json.loads(text)
        if isinstance(json_data, list) and json_data:
            education_info = []
            for edu in json_data:
                edu_str = ""
                if "instituteName" in edu:
                    edu_str += edu["instituteName"]
                if "faculty" in edu:
                    edu_str += ", факультет: " + edu["faculty"]
                if "speciality" in edu:
                    edu_str += ", специальность: " + edu["speciality"]
                if "qualification" in edu:
                    edu_str += ", квалификация: " + edu["qualification"]
                if "graduateYear" in edu:
                    edu_str += ", год окончания: " + str(edu["graduateYear"])
                education_info.append(edu_str)

            joined_info = "; ".join(education_info).strip()
            if joined_info:
                return joined_info
            else:
                return np.nan
    except json.JSONDecodeError:
        return np.nan
    return np.nan


def map_workExperienceList(text):
    if pd.isna(text):
        return np.nan
    try:
        json_data = json.loads(text)
        if isinstance(json_data, list) and json_data:
            work_experience_info = []
            for experience in json_data:
                experience_str = ""
                if "companyName" in experience:
                    experience_str += experience["companyName"]
                if "jobTitle" in experience:
                    experience_str += ", должность: " + experience["jobTitle"]
                if "dateFrom" in experience and "dateTo" in experience:
                    experience_str += ", период работы: {} - {}".format(
                        experience["dateFrom"][:10], experience["dateTo"][:10]
                    )
                if "demands" in experience:
                    demands_cleaned = clean_html(experience["demands"])
                    experience_str += ", обязанности: " + demands_cleaned
                work_experience_info.append(experience_str)

            return "; ".join(work_experience_info)
    except json.JSONDecodeError:
        return np.nan
    return np.nan


def map_otherCertificates(text):
    if (
        pd.isna(text)
        or not text.strip()
        or text
        in [
            '<p><br-data-mce-bogus="1"></p>',
            "<p></p>-<p></p>",
            "<ol>-<li></li>-</ol>",
            "[]",
        ]
    ):
        return np.nan
    soup = BeautifulSoup(text, "html.parser")
    cleaned_str = soup.get_text(separator="; ")
    cleaned_str = cleaned_str.replace("-", " ").strip()

    if cleaned_str:
        return cleaned_str
    else:
        return np.nan


def map_additionalEducationList(text):
    if pd.isna(text):
        return np.nan
    try:
        json_data = json.loads(text)
        if isinstance(json_data, list) and json_data:
            additional_education_info = []
            for course in json_data:
                course_str = ""
                if "organization" in course:
                    course_str += course["organization"]
                if "graduateYear" in course:
                    course_str += ", год окончания: " + str(course["graduateYear"])
                additional_education_info.append(course_str)

            return "; ".join(additional_education_info)
    except json.JSONDecodeError:
        return np.nan
    return np.nan


def map_birthday(birthday):
    if pd.isna(birthday):
        return np.nan
    try:
        birth_date = pd.to_datetime(birthday)
        if birth_date.year < 1900 or birth_date.year > 2023:
            return np.nan
        return birth_date.strftime("%Y-%m-%d")
    except:
        return np.nan


def map_worldskills(json_str):
    if pd.isna(json_str) or json_str == "[]":
        return np.nan
    try:
        json_data = json.loads(json_str)
        if isinstance(json_data, list) and json_data:
            skills = [
                skill["russianName"] for skill in json_data if "russianName" in skill
            ]
            if skills:
                return "; ".join(skills)
    except json.JSONDecodeError:
        return np.nan
    return np.nan

## Resume


In [244]:
chunksize = 10000
num_samples = NROWS
df_resume = pd.DataFrame()

for chunk in pd.read_csv(
    "../datasets/cv.csv",
    chunksize=chunksize,
    on_bad_lines="skip",
    sep="|",
    low_memory=False,
):
    sample_size = min(len(chunk), int(num_samples / (5000000 / chunksize)))
    df_resume = pd.concat([df_resume, chunk.sample(n=sample_size)])
    num_samples -= sample_size
    if num_samples <= 0:
        break
df_resume.head()

,id,candidateId,stateRegionCode,locality,localityName,birthday,gender,age,positionName,dateCreate,...,softSkills,workExperienceList,scheduleType,salary,busyType,retrainingCapability,businessTrip,languageKnowledge,relocation,innerInfo
9595,15f93170-f961-11ed-967f-e57b87a63fa7,b27f90a0-f95e-11ed-b3cf-7dbf5d2a6230,6100000000000,6100000000000,Ростовская-область,2004-07-20T16:00:00+0400,Мужской,19.0,Подсобный-рабочий,2023-05-23T14:58:09+0300,...,[],[],Неполный-рабочий-день,16242.0,NaN,Не-готов-к-переобучению,Не-готов-к-командировкам,"[{""codeLanguage"": ""Русский"", ""level"": ""Родной""...",Не-готов-к-переезду,"{""idUser"": ""689fb2b0-249d-11e5-b38b-1ff7059456..."
4687,03d80f30-a6dc-11ec-8605-1b29d3b53cbb,aeb93ec0-a41f-11ec-86cb-d768660431c4,5200000000000,5200400100000,"Нижегородская-область,-г.-Балахна",1971-01-20T00:00:00+0300,Мужской,52.0,водитель.-мастер-инструктор-по-вождению.мастер...,2022-03-18T19:54:02+0300,...,[],"[{""achievements"": ""<p>водительсое удостоверени...",Сменный-график,35000.0,Полная-занятость,Готов-к-переобучению,Не-готов-к-командировкам,[],Не-готов-к-переезду,"{""idUser"": ""390aaa90-1995-11e5-8d85-1ff7059456..."
1963,0386fd30-5ef9-11ee-b704-c7bd3a8c3ec7,448c1bf0-5ef7-11ee-b250-e73e8fa159bf,7800000000000,7800000000000,г-Санкт-Петербург,2006-09-28T16:00:00+0400,Женский,17.0,Временное-устройство-несовершеннолетних,2023-09-29T21:50:09+0300,...,[],[],Неполный-рабочий-день,10000.0,NaN,Не-готов-к-переобучению,Не-готов-к-командировкам,"[{""codeLanguage"": ""Русский"", ""level"": ""Родной""...",Не-готов-к-переезду,"{""idUser"": ""7734c0f0-9472-11ea-a9ed-195c1cfba7..."
4450,03d9f8f0-6140-11eb-8ec9-3bfa22f2d66b,872779f0-613e-11eb-8ec9-3bfa22f2d66b,7100000000000,7100000000000,Тульская-область,1993-02-08T00:00:00+0300,Мужской,30.0,начальник-смены,2021-01-28T11:08:33+0300,...,[],"[{""companyName"": ""ООО \""МэнпауэрГрупп\"""", ""dat...","Сменный-график,Полный-рабочий-день",40000.0,Полная-занятость,Готов-к-переобучению,Готов-к-командировкам,[],Готов-к-переезду,"{""idUser"": ""4431c3a0-2c86-11e5-9830-1ff7059456..."
9238,06a962f4-21de-11e7-8a20-736ab11edb0c,c5d42010-21da-11e7-8a20-736ab11edb0c,3900000000000,3900000100000,"Калининградская-область,-г.-Калининград",1964-04-12T00:00:00+0300,Мужской,59.0,Менеджер-проекта,2017-04-15T16:18:33+0300,...,[],"[{""achievements"": ""<p>Компания является лидеро...",Полный-рабочий-день,1000.0,Полная-занятость,Готов-к-переобучению,Готов-к-командировкам,"[{""codeLanguage"": ""Английский"", ""level"": ""Спос...",Готов-к-переезду,"{""idUser"": ""d5f4b4d0-02ad-11e5-85f7-1ff7059456..."


In [245]:
df_resume["localityName"] = df_resume["localityName"].apply(clean_divis)  # Место жительства
df_resume["relocation"] = df_resume["relocation"].apply(clean_divis)  # Готовность к переезду
df_resume["businessTrip"] = df_resume["businessTrip"].apply(clean_divis)  # Готовность к командировкам
df_resume["retrainingCapability"] = df_resume["retrainingCapability"].apply(clean_divis)  # Готовность к переобучению
df_resume["busyType"] = df_resume["busyType"].apply(clean_divis)  # Тип занятости
df_resume["scheduleType"] = df_resume["scheduleType"].apply(clean_divis)  # Желаемый график работы
df_resume["gender"] = df_resume["gender"].apply(clean_divis)  # Пол
df_resume["volunteersInspectionStatus"] = df_resume["volunteersInspectionStatus"].apply(clean_divis)  # Статус участия в движении DOBRO.RU
df_resume["narkInspectionStatus"] = df_resume["narkInspectionStatus"].apply(clean_divis)  # Статус проверки квалификации НАРК
df_resume["positionName"] = df_resume["positionName"].apply(clean_divis)  # Желаемая должность
df_resume["academicDegree"] = df_resume["academicDegree"].apply(clean_divis)  # Ученая степень
df_resume["abilympicsParticipation"] = df_resume["abilympicsParticipation"].apply(clean_divis)  # Участие в движении Абилимпикс
df_resume["salary"] = df_resume["salary"].apply(map_digits)  # Желаемая зарплата
df_resume["experience"] = df_resume["experience"].apply(map_digits)  # Опыт работы
df_resume["age"] = df_resume["age"].apply(map_digits)  # Возраст
df_resume["country"] = df_resume["country"].apply(map_country)  # Страна проживания
df_resume["languageKnowledge"] = df_resume["languageKnowledge"].apply(map_languageKnowledgeResume)  # Знание языков
df_resume["driveLicenses"] = df_resume["driveLicenses"].apply(map_driveLicenses)  # Водительские права
df_resume["hardSkills"] = df_resume["hardSkills"].apply(map_hardSkillsResume)  # Профессиональные навыки
df_resume["softSkills"] = df_resume["softSkills"].apply(map_softSkillsResume)  # Гибкие навыки
df_resume["educationList"] = df_resume["educationList"].apply(map_educationList)  # Образование
df_resume["workExperienceList"] = df_resume["workExperienceList"].apply(map_workExperienceList)  # Информация об опыте работы
df_resume["otherCertificates"] = df_resume["otherCertificates"].apply(map_otherCertificates)  # Другие сертификаты
df_resume["additionalEducationList"] = df_resume["additionalEducationList"].apply(map_additionalEducationList)  # Дополнительное образование
df_resume["birthday"] = df_resume["birthday"].apply(map_birthday)  # Дата рождения
df_resume["worldskills"] = df_resume["worldskills"].apply(map_worldskills)  # Worldskills компетенции

/var/folders/26/jypvmr_s4v10dwrm3dtpx1gm0000gn/T/ipykernel_97676/3280539921.py:4: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(text, "html.parser")


In [246]:
def create_resume_text(row):
    parts = []

    # Базовая информация
    if not pd.isna(row["positionName"]):
        parts.append(row["positionName"])
        
    # Опыт работы
    if not pd.isna(row["workExperienceList"]):
        parts.append(f"Опыт работы: {row['workExperienceList']}")

    # Образование и курсы
    if not pd.isna(row["educationList"]):
        parts.append(f"Образование: {row['educationList']}")
    if not pd.isna(row["additionalEducationList"]):
        parts.append(f"Дополнительное образование: {row['additionalEducationList']}")

    # Профессиональные и гибкие навыки
    if not pd.isna(row["hardSkills"]):
        parts.append(f"Профессиональные навыки: {row['hardSkills']}")
    if not pd.isna(row["softSkills"]):
        parts.append(f"Гибкие навыки: {row['softSkills']}")

    # Дополнительная информация
    if not pd.isna(row["otherCertificates"]):
        parts.append(f"Сертификаты и достижения: {row['otherCertificates']}")
    if not pd.isna(row["worldskills"]):
        parts.append(f"Worldskills компетенции: {row['worldskills']}")

    return " | ".join(parts)


resume_texts = df_resume.apply(create_resume_text, axis=1).tolist()

## Job


In [247]:
df_job = pd.read_csv(
    "../datasets/vacancy.csv",
    encoding="UTF-8",
    on_bad_lines="skip",
    sep="|",
    low_memory=False,
)

random_indices = np.random.choice(df_job.shape[0], NROWS, replace=False)
df_job = df_job.iloc[random_indices]

df_job.head()

,id,academic_degree,accommodation_capability,accommodation_type,additional_premium,additional_requirements,bonus_type,measure_type,busy_type,career_perspective,...,federalDistrictCode,industryBranchName,contactList,company_name,full_company_name,company_inn,company,languageKnowledge,hardSkills,softSkills
510405,e5e6bb62-086b-11ed-bddd-bf2cfe8c828d,NaN,False,NaN,NaN,Тяжелые и вредные условия труда.<br/>Лаборант ...,NaN,NaN,Полная занятость,False,...,3.0,NaN,NaN,NaN,"ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ ""РН-К...",2.309095e+09,"{""companycode"":""1052304983785"",""email"":""mail@r...",[],[],[]
317662,3b1b73b5-5438-11ed-a384-791a818bdadb,NaN,False,NaN,NaN,NaN,NaN,NaN,Полная занятость,False,...,5.0,NaN,NaN,NaN,"Акционерное общество ""Тандер"" филиал в г. Екат...",2.310031e+09,"{""companycode"":""c2c67820-74b9-11ea-8bf8-736ab1...",[],[],[]
439332,30ec9282-c298-11eb-9dd6-bf2cfe8c828d,NaN,False,NaN,NaN,Тяжелые и вредные условия труда.<br/>Ответстве...,NaN,NaN,Полная занятость,False,...,4.0,NaN,NaN,NaN,ГОСУДАРСТВЕННОЕ БЮДЖЕТНОЕ УЧРЕЖДЕНИЕ ЗДРАВООХР...,6.316044e+09,"{""companycode"":""1036300553970"",""email"":""soptd@...",[],[],[]
499751,4965b436-a907-11ec-a36c-550ed7335bbe,NaN,False,NaN,NaN,NaN,NaN,NaN,Частичная занятость,False,...,3.0,NaN,NaN,NaN,МУНИЦИПАЛЬНОЕ БЮДЖЕТНОЕ ОБЩЕОБРАЗОВАТЕЛЬНОЕ УЧ...,9.106007e+09,"{""companycode"":""1149102169765"",""email"":""shkola...","[{""code_language"":""Русский"",""id_owner"":""4965b4...",[],[]
96702,64c71e06-1f02-11ed-b6f7-35178352af9b,NaN,True,DORMITORY,10.0,<p><strong>Условия:</strong></p> <ul> <li>офиц...,MONTHLY,PERCENT,Полная занятость,False,...,3.0,NaN,NaN,NaN,"Общество с ограниченной ответственностью ""Комп...",7.702348e+09,"{""companycode"":""fdccb420-8bd8-11eb-a83d-3bfa22...","[{""code_language"":""Английский"",""id_owner"":""64c...",[],[]


In [248]:
df_job["position_responsibilities"] = df_job["position_responsibilities"].apply(clean_html)  # Обязанности на должности
df_job["position_requirements"] = df_job["position_requirements"].apply(clean_html)  # Требования к должности
df_job["additional_requirements"] = df_job["additional_requirements"].apply(clean_html)  # Дополнительные требования
df_job["other_vacancy_benefit"] = df_job["other_vacancy_benefit"].apply(clean_html)  # Другие льготы вакансии
df_job["softSkills"] = df_job["softSkills"].apply(map_softSkillsJob)  # Гибкие навыки
df_job["hardSkills"] = df_job["hardSkills"].apply(map_hardSkillsJob)  # Профессиональные навыки
df_job["required_drive_license"] = df_job["required_drive_license"].apply(map_driveLicenses)  # Требуемые водительские права
df_job["languageKnowledge"] = df_job["languageKnowledge"].apply(map_languageKnowledgeJob)  # Знание иностранных языков
df_job["accommodation_capability"] = df_job["accommodation_capability"].apply(map_boolean_to_yes_no)  # Возможность предоставления жилья
df_job["career_perspective"] = df_job["career_perspective"].apply(map_boolean_to_yes_no)  # Перспективы карьерного роста'
df_job["is_quoted"] = df_job["is_quoted"].apply(map_boolean_to_yes_no)  # Квотируемая вакансия
df_job["retraining_capability"] = df_job["retraining_capability"].apply(map_boolean_to_yes_no)  # Возможность переобучения
df_job["is_mobility_program"] = df_job["is_mobility_program"].apply(map_boolean_to_yes_no)  # Программа мобильности
df_job["need_medcard"] = df_job["need_medcard"].apply(map_boolean_to_yes_no)  # Необходимость медицинской книжки
df_job["is_uzbekistan_recruitment"] = df_job["is_uzbekistan_recruitment"].apply(map_boolean_to_yes_no)  # Возможность найма граждан Узбекистана
df_job["accommodation_type"] = df_job["accommodation_type"].apply(map_accommodation_type)  # Тип предоставляемого жилья
df_job["company_business_size"] = df_job["company_business_size"].apply(map_company_business_size)  # Размер компании
df_job["transport_compensation"] = df_job["transport_compensation"].apply(map_transport_compensation)  # Компенсация транспортных расходов
df_job["salary_min"] = df_job["salary_min"].apply(map_digits)  # Минимальная заработная плата
df_job["salary_max"] = df_job["salary_max"].apply(map_digits)  # Максимальная заработная плата
df_job["additional_premium"] = df_job["additional_premium"].apply(map_digits)  # Дополнительная премия

# 'full_company_name': 'Полное название компании'
# 'vacancy_address': 'Адрес вакансии'
# 'education': 'Образование'
# 'education_speciality': 'Специальность по образованию'
# 'required_certificates': 'Требуемые сертификаты'
# 'professionalSphereName': 'Название профессиональной сферы'
# 'industryBranchName': 'Название отрасли промышленности'
# 'academic_degree': 'Ученая степень'
# 'vacancy_name': 'Наименование вакансии'
# 'required_experience': 'Требуемый опыт работы
# 'busy_type': 'Тип занятости'
# 'schedule_type': 'Тип графика работы'

/var/folders/26/jypvmr_s4v10dwrm3dtpx1gm0000gn/T/ipykernel_97676/3280539921.py:4: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(text, "html.parser")


In [249]:
def create_job_text(row):
    parts = []

    # Job Details
    if not pd.isna(row["vacancy_name"]):
        parts.append(f"{row['vacancy_name']}")

    if not pd.isna(row["position_responsibilities"]):
        parts.append(f"Обязанности: {row['position_responsibilities']}")
    if not pd.isna(row["position_requirements"]):
        parts.append(f"Требования: {row['position_requirements']}")
    if not pd.isna(row["additional_requirements"]):
        parts.append(f"Дополнительные требования: {row['additional_requirements']}")

    if not pd.isna(row["hardSkills"]):
        parts.append(f"Профессиональные навыки: {row['hardSkills']}")
    if not pd.isna(row["softSkills"]):
        parts.append(f"Гибкие навыки: {row['softSkills']}")

    return " | ".join(parts)


job_texts = df_job.apply(create_job_text, axis=1).tolist()

## Text Preprocessing


In [250]:
# Расширенный список стоп-слов
sw = stopwords.words("russian")
additional_sw = "мои оно мной мною мог могут мор мое мочь оба нам нами ними однако нему никуда наш нею неё наша наше наши очень отсюда вон вами ваш ваша ваше ваши весь всем всеми вся ими ею будем будете будешь буду будь будут кому кого которой которого которая которые который которых кем каждое каждая каждые каждый кажется та те тому собой тобой собою тобою тою хотеть хочешь свое свои твой своей своего своих твоя твоё сама сами теми само самом самому самой самого самим самими самих саму чему тебе такое такие также такая сих тех ту эта это этому туда этим этими этих абы аж ан благо буде вроде дабы едва ежели затем зато ибо итак кабы коли коль либо лишь нежели пока покамест покуда поскольку притом причем пускай пусть ровно сиречь словно также точно хотя чисто якобы "
pronouns = "я мы ты вы он она оно они себя мой твой ваш наш свой его ее их то это тот этот такой таков столько весь всякий сам самый каждый любой иной другой кто что какой каков чей сколько никто ничто некого нечего никакой ничей нисколько кто-то кое-кто кто-нибудь кто-либо что-то кое-что что-нибудь что-либо какой-то какой-либо какой-нибудь некто нечто некоторый некий"
conjunctions = "что чтобы как когда ибо пока будто словно если потому что оттого что так как так что лишь только как будто с тех пор как в связи с тем что для того чтобы кто как когда который какой где куда откуда"
digits = "ноль один два три четыре пять шесть семь восемь девять десять одиннадцать двенадцать тринадцать четырнадцать пятнадцать шестнадцать семнадцать восемнадцать девятнадцать двадцать тридцать сорок пятьдесят шестьдесят семьдесят восемьдесят девяносто сто"
modal_words = "вероятно возможно видимо по-видимому кажется наверное безусловно верно  действительно конечно несомненно разумеется"
particles = "да так точно ну да не ни неужели ли разве а что ли что за то-то как ну и ведь даже еще ведь уже все все-таки просто прямо вон это вот как словно будто точно как будто вроде как бы именно как раз подлинно ровно лишь только хоть всего исключительно вряд ли едва ли"
prepositions = "близ  вблизи  вдоль  вокруг  впереди  внутрь  внутри  возле  около  поверх  сверху  сверх  позади  сзади  сквозь  среди  прежде  мимо  вслед  согласно  подобно  навстречу  против  напротив  вопреки  после  кроме  вместе  вдали  наряду  совместно  согласно  нежели вроде от бишь до без аж тех раньше совсем только итак например из прямо ли следствие а поскольку благо пускай благодаря случае затем притом также связи время при чтоб просто того невзирая даром вместо точно покуда тогда зато ради ан буде прежде насчет раз причине тому так даже исходя коль кабы более ровно либо помимо как-то будто если словно лишь бы и не будь пор тоже разве чуть как хотя наряду потому пусть в равно между сверх ибо на судя то чтобы относительно или счет за но сравнению причем оттого есть когда уж ввиду тем для дабы чем хоть с вплоть скоро едва после той да вопреки ежели кроме сиречь же коли под абы несмотря все пока покамест паче прямо-таки перед что по вдруг якобы подобно"
evaluative = "наиболее наименее лучший больший высший низший худший более менее"


sw.extend(additional_sw.split())
sw.extend(pronouns.split())
sw.extend(conjunctions.split())
sw.extend(digits.split())
sw.extend(modal_words.split())
sw.extend(particles.split())
sw.extend(prepositions.split())
sw.extend(evaluative.split())
sw = list(set(sw))

In [251]:
pattern = re.compile("[^а-яА-Яa-zA-Z0-9\-.,;]+")
morph = pymorphy3.MorphAnalyzer()

def tokenize_and_preprocess(text, stopwords):
    text = text.lower()
    text = pattern.sub(" ", text)
    tokens = simple_preprocess(text, deacc=True)
    tokens = [
        morph.parse(token)[0].normal_form
        for token in tokens
        if token not in stopwords and len(token) > 2
    ]
    return tokens

In [252]:
%%time

processed_resume_texts = [tokenize_and_preprocess(text, sw) for text in resume_texts]

CPU times: user 1min 10s, sys: 301 ms, total: 1min 11s
Wall time: 1min 11s


In [253]:
%%time

processed_job_texts = [tokenize_and_preprocess(text, sw) for text in job_texts]

CPU times: user 1min 42s, sys: 714 ms, total: 1min 42s
Wall time: 1min 43s


# Models

## Doc2Vec

In [254]:
tagged_data = [
    TaggedDocument(words=text, tags=[f"resume_{i}"])
    for i, text in enumerate(processed_resume_texts)
]
tagged_data += [
    TaggedDocument(words=text, tags=[f"job_{i}"])
    for i, text in enumerate(processed_job_texts)
]

In [255]:
model = Doc2Vec(
    vector_size=DOC2VEC_VECTOR_SIZE,
    min_count=DOC2VEC_MIN_COUNT,
    epochs=DOC2VEC_EPOCHS,
    alpha=DOC2VEC_ALPHA,
)
model.build_vocab(tagged_data)

keys = model.wv.key_to_index.keys()
len(keys) # length of vocabulary keys

14095

In [256]:
%%time

for epoch in range(model.epochs):
    print(f"Training epoch {epoch+1}/{model.epochs}")
    model.train(tagged_data, total_examples=model.corpus_count, epochs=model.epochs)

model.save(DOC2VEC_MODEL_PATH)
print("Model saved")

Training epoch 1/100
Training epoch 2/100


### Visualisation

In [ ]:
def plot_embeddings(model, tagged_docs):
    vectors = [model.infer_vector(doc.words) for doc in tagged_docs]
    labels = [doc.tags[0] for doc in tagged_docs]
    vectors_np = np.array(vectors)
    tsne = TSNE(n_components=2, random_state=RANDOM_STATE)
    vectors_2d = tsne.fit_transform(vectors_np)

    traces = []
    for point, label in zip(vectors_2d, labels):
        color = 'red' if 'resume' in label else 'blue'
        trace = go.Scatter(
            x=[point[0]], 
            y=[point[1]],
            mode='markers+text',
            marker=dict(color=color),
            text=label,
            textposition="bottom center"
        )
        traces.append(trace)

    layout = go.Layout(title="Embeddings Visualization", showlegend=False)
    fig = go.Figure(data=traces, layout=layout)

    fig.show()

In [ ]:
plot_embeddings(model, tagged_data[:1000])

## BERT

In [ ]:
bert_model = BertModel.from_pretrained(BERT_MODEL_NAME)
tokenizer = BertTokenizer.from_pretrained(BERT_MODEL_NAME)

Some weights of the model checkpoint at DeepPavlov/rubert-base-cased were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
def get_bert_embeddings(texts, bert_model, tokenizer):
    embeddings = []
    for text in texts:
        encoded_input = tokenizer(text, return_tensors='pt', padding=True, truncation=True, max_length=BERT_MAX_LENGTH)
        with torch.no_grad():
            model_output = bert_model(**encoded_input)
        emb = model_output.last_hidden_state.mean(dim=1)
        if emb.shape[-1] != 768:
            emb = np.zeros((1, 768))
        else:
            emb = emb.detach().cpu().numpy()
            emb = np.squeeze(emb)
            if emb.shape[0] != 768:
                emb = np.zeros(768)
        embeddings.append(emb)
    return np.array(embeddings)

## Doc2Vec + BERT

In [ ]:

def combine_embeddings(doc2vec_model, bert_embeddings, texts):
    combined_embeddings = []
    for bert_emb, text in zip(bert_embeddings, texts):
        doc2vec_model.random.seed(SEED)
        doc2vec_emb = doc2vec_model.infer_vector(text.split())
        doc2vec_emb = np.atleast_1d(doc2vec_emb)
        bert_emb = np.atleast_1d(bert_emb)
        
        doc2vec_emb /= np.linalg.norm(doc2vec_emb, ord=2) + 1e-10
        bert_emb /= np.linalg.norm(bert_emb, ord=2) + 1e-10

        if doc2vec_emb.shape[0] != 50 or bert_emb.shape[0] != 768:
            continue

        combined_emb = np.concatenate((doc2vec_emb, bert_emb), axis=0)
        combined_embeddings.append(combined_emb)
    return combined_embeddings

# Ranking

In [ ]:
items = [
    "Опытный системный аналитик с пятилетним стажем работы в крупной IT-компании. Основные обязанности включали анализ бизнес-процессов, сбор требований от заказчиков, разработку технических заданий и координацию разработчиков. Умею работать с SQL, UML, BPMN. Опыт работы с Agile и Scrum. В прошлом году успешно реализовал проект автоматизации процессов для крупного ритейлера, что позволило клиенту увеличить эффективность работы на 20%. Отличные коммуникативные навыки, способность работать в команде и под высоким давлением.",
    "Менеджер по продажам с более чем десятилетним опытом в сфере B2B и B2C. Специализируюсь на продажах в области финансовых услуг и страхования. Отлично разбираюсь в продуктах компании, умею находить подход к различным типам клиентов. Имею опыт организации и проведения презентаций, участия в переговорах и заключения крупных сделок. В прошлом году лично привлек более 30 новых корпоративных клиентов, что способствовало росту общего оборота компании на 15%. Владею техниками активных продаж, хорошо ориентируюсь в CRM-системах.",
    "Главный инженер с более чем 15-летним опытом работы в сфере машиностроения. Ответственен за разработку и внедрение новых технологий, управление инженерными командами, контроль качества продукции. Имею опыт работы с CAD/CAM системами, знаком с современными методами производства и автоматизации. На моем счету множество успешно реализованных проектов, в том числе разработка и запуск новой линии по производству автомобильных компонентов. Отличные лидерские качества, способность принимать быстрые и эффективные решения в критических ситуациях.",
    "Веб-разработчик с опытом работы более 8 лет. Специализируюсь на front-end и back-end разработке. Владею HTML, CSS, JavaScript, а также фреймворками React и Node.js. Работал над созданием и поддержкой нескольких крупных веб-платформ, включая интернет-магазины и корпоративные сайты. Имею опыт работы в Agile-командах, умею четко следовать срокам и требованиям заказчика. За последний год разработал и внедрил новый интерфейс для онлайн-магазина, что привело к увеличению конверсии на 30%.",
    "Творческий и инновационный графический дизайнер с 7-летним опытом работы в рекламных агентствах и дизайн-студиях. Специализируюсь на создании брендинга, рекламных материалов, веб-дизайна и UX/UI. Владею Adobe Photoshop, Illustrator, InDesign. Мои работы отличаются оригинальностью и уникальностью, я всегда стремлюсь найти нестандартные решения для задач клиентов. Разработал брендинг и фирменный стиль для нескольких стартапов, что помогло им выделиться на рынке и привлечь новых клиентов. Умею работать в сжатые сроки, легко адаптируюсь к новым требованиям и тенденциям в дизайне."
]

jobs = [
    "Требуется опытный руководитель отдела продаж для работы в крупной торговой компании. Обязанности: разработка и реализация стратегии продаж, управление командой менеджеров, анализ рынка и конкурентов, участие в переговорах на высоком уровне. Требования: опыт работы на аналогичной должности не менее 5 лет, отличные навыки управления и коммуникации, знание современных технологий продаж. Предлагаем стабильную работу в динамично развивающейся компании, конкурентоспособную заработную плату и возможности для профессионального роста.",
    "ИТ-компания ищет талантливого программиста 1C для разработки и поддержки корпоративных приложений. Основные задачи: разработка и оптимизация программных решений на платформе 1C, работа с клиентами по техническим заданиям, тестирование и документирование программного обеспечения. Требуется знание 1C:Предприятие 8, опыт программирования и понимание бизнес-процессов. Мы предлагаем работу в комфортабельном офисе, дружный коллектив и достойную заработную плату.",
    "В маркетинговое агентство требуется креативный и амбициозный маркетолог. Обязанности: разработка маркетинговых стратегий, проведение рекламных кампаний, анализ эффективности маркетинговых действий, работа с социальными сетями и контентом. Требуется опыт в маркетинге не менее 3 лет, умение работать в команде, знание инструментов интернет-маркетинга. Мы предлагаем работу в динамичной среде, возможность реализовать свои идеи и конкурентоспособную заработную плату.",
    "Требуется инженер-конструктор в инженерно-проектное бюро. Задачи: разработка проектной документации, расчеты и моделирование в CAD-системах, сопровождение проектов на всех этапах. Требования: высшее техническое образование, опыт работы в проектировании, знание AutoCAD и других CAD-систем. Предлагаем интересные проекты, работу в команде профессионалов, стабильную заработную плату и возможности для карьерного роста.",
    "В рекламное агентство требуется творческий копирайтер для создания текстов различной тематики. Обязанности: написание рекламных и PR-текстов, создание контента для сайтов и социальных сетей, работа над креативными концепциями. Требуется опыт написания текстов, креативность, отличное владение русским языком. Мы предлагаем возможность реализации творческого потенциала, работу над интересными проектами и достойную оплату труда."
]

In [ ]:
def preprocess_text(text):
    text = text.lower()
    pattern = re.compile("[^а-яА-Яa-zA-Z0-9\-.,;]+")
    text = pattern.sub(" ", text)
    text = ' '.join(text.split())
    return text

## Doc2Vec

In [ ]:
def rank_items_doc2vec(doc2vec_model, desc, items, threshold=0.5):
    desc = preprocess_text(desc)
    doc2vec_model.random.seed(SEED)
    desc_vector = doc2vec_model.infer_vector(desc.split()).reshape(1, -1)
    ranked_items = []
    for item in items:
        item = preprocess_text(item)
        doc2vec_model.random.seed(SEED)
        item_vector = model.infer_vector(item.split()).reshape(1, -1)
        similarity = cosine_similarity(desc_vector, item_vector)[0][0]
        if similarity >= threshold:
            ranked_items.append((item, similarity))
    ranked_items.sort(key=lambda x: x[1], reverse=True)
    return [(item, round(similarity * 100, 2)) for item, similarity in ranked_items]

### Example

In [ ]:
doc2vec_model = Doc2Vec.load(DOC2VEC_MODEL_PATH)

In [ ]:
rank_items_doc2vec(doc2vec_model, jobs[3], items) # вакансия веб-разработчика

[('главный инженер с более чем 15-летним опытом работы в сфере машиностроения. ответственен за разработку и внедрение новых технологий, управление инженерными командами, контроль качества продукции. имею опыт работы с cad cam системами, знаком с современными методами производства и автоматизации. на моем счету множество успешно реализованных проектов, в том числе разработка и запуск новой линии по производству автомобильных компонентов. отличные лидерские качества, способность принимать быстрые и эффективные решения в критических ситуациях.',
  72.58),
 ('веб-разработчик с опытом работы более 8 лет. специализируюсь на front-end и back-end разработке. владею html, css, javascript, а также фреймворками react и node.js. работал над созданием и поддержкой нескольких крупных веб-платформ, включая интернет-магазины и корпоративные сайты. имею опыт работы в agile-командах, умею четко следовать срокам и требованиям заказчика. за последний год разработал и внедрил новый интерфейс для онлайн-маг

In [ ]:
rank_items_doc2vec(doc2vec_model, items[4], jobs) # резюме дизайнера

[('в рекламное агентство требуется творческий копирайтер для создания текстов различной тематики. обязанности написание рекламных и pr-текстов, создание контента для сайтов и социальных сетей, работа над креативными концепциями. требуется опыт написания текстов, креативность, отличное владение русским языком. мы предлагаем возможность реализации творческого потенциала, работу над интересными проектами и достойную оплату труда.',
  53.09)]

## BERT

In [ ]:
def rank_items_bert(bert_model, tokenizer, desc, items, threshold=0.5):
    desc = preprocess_text(desc)
    desc_bert_emb = get_bert_embeddings([desc], bert_model, tokenizer)[0]
    items_bert_embs = get_bert_embeddings(items, bert_model, tokenizer)
    similarities = cosine_similarity([desc_bert_emb], items_bert_embs)[0]
    ranked_items = [(item, similarity) for item, similarity in zip(items, similarities) if similarity >= threshold]
    ranked_items.sort(key=lambda x: x[1], reverse=True)
    return [(item, round(similarity * 100, 2)) for item, similarity in ranked_items]

### Example

In [ ]:
bert_model = BertModel.from_pretrained(BERT_MODEL_NAME)
tokenizer = BertTokenizer.from_pretrained(BERT_MODEL_NAME)

Some weights of the model checkpoint at DeepPavlov/rubert-base-cased were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
rank_items_bert(bert_model, tokenizer, jobs[3], items) # Ранжирование резюме для вакансии веб-разработчика

[('Опытный системный аналитик с пятилетним стажем работы в крупной IT-компании. Основные обязанности включали анализ бизнес-процессов, сбор требований от заказчиков, разработку технических заданий и координацию разработчиков. Умею работать с SQL, UML, BPMN. Опыт работы с Agile и Scrum. В прошлом году успешно реализовал проект автоматизации процессов для крупного ритейлера, что позволило клиенту увеличить эффективность работы на 20%. Отличные коммуникативные навыки, способность работать в команде и под высоким давлением.',
  84.21),
 ('Главный инженер с более чем 15-летним опытом работы в сфере машиностроения. Ответственен за разработку и внедрение новых технологий, управление инженерными командами, контроль качества продукции. Имею опыт работы с CAD/CAM системами, знаком с современными методами производства и автоматизации. На моем счету множество успешно реализованных проектов, в том числе разработка и запуск новой линии по производству автомобильных компонентов. Отличные лидерские ка

In [ ]:
rank_items_bert(bert_model, tokenizer, items[4], jobs) # Ранжирование вакансий для резюме дизайнера

[('В рекламное агентство требуется творческий копирайтер для создания текстов различной тематики. Обязанности: написание рекламных и PR-текстов, создание контента для сайтов и социальных сетей, работа над креативными концепциями. Требуется опыт написания текстов, креативность, отличное владение русским языком. Мы предлагаем возможность реализации творческого потенциала, работу над интересными проектами и достойную оплату труда.',
  82.77),
 ('В маркетинговое агентство требуется креативный и амбициозный маркетолог. Обязанности: разработка маркетинговых стратегий, проведение рекламных кампаний, анализ эффективности маркетинговых действий, работа с социальными сетями и контентом. Требуется опыт в маркетинге не менее 3 лет, умение работать в команде, знание инструментов интернет-маркетинга. Мы предлагаем работу в динамичной среде, возможность реализовать свои идеи и конкурентоспособную заработную плату.',
  80.53),
 ('ИТ-компания ищет талантливого программиста 1C для разработки и поддержки

## Doc2Vec + BERT

In [ ]:
def rank_items_doc2vec_bert(doc2vec_model, bert_model, tokenizer, desc, items, threshold=0.5):
    desc = preprocess_text(desc)
    desc_bert_embs = get_bert_embeddings([desc], bert_model, tokenizer)
    desc_vector = combine_embeddings(doc2vec_model, desc_bert_embs, [desc])[0]

    items_bert_embs = get_bert_embeddings(items, bert_model, tokenizer)
    combined_new_resume_embs = combine_embeddings(doc2vec_model, items_bert_embs, items)

    similarities = cosine_similarity([desc_vector], combined_new_resume_embs)[0]
    ranked_items = [(item, similarity) for item, similarity in zip(items, similarities) if similarity >= threshold]
    ranked_items.sort(key=lambda x: x[1], reverse=True)

    return [(item, round(similarity * 100, 2)) for item, similarity in ranked_items]

### Example

In [ ]:
doc2vec_model = Doc2Vec.load(DOC2VEC_MODEL_PATH)
bert_model = BertModel.from_pretrained(BERT_MODEL_NAME)
tokenizer = BertTokenizer.from_pretrained(BERT_MODEL_NAME)

Some weights of the model checkpoint at DeepPavlov/rubert-base-cased were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
rank_items_doc2vec_bert(doc2vec_model, bert_model, tokenizer, jobs[3], items) # вакансия веб-разработчика

[('Опытный системный аналитик с пятилетним стажем работы в крупной IT-компании. Основные обязанности включали анализ бизнес-процессов, сбор требований от заказчиков, разработку технических заданий и координацию разработчиков. Умею работать с SQL, UML, BPMN. Опыт работы с Agile и Scrum. В прошлом году успешно реализовал проект автоматизации процессов для крупного ритейлера, что позволило клиенту увеличить эффективность работы на 20%. Отличные коммуникативные навыки, способность работать в команде и под высоким давлением.',
  78.36),
 ('Главный инженер с более чем 15-летним опытом работы в сфере машиностроения. Ответственен за разработку и внедрение новых технологий, управление инженерными командами, контроль качества продукции. Имею опыт работы с CAD/CAM системами, знаком с современными методами производства и автоматизации. На моем счету множество успешно реализованных проектов, в том числе разработка и запуск новой линии по производству автомобильных компонентов. Отличные лидерские ка

In [ ]:
rank_items_doc2vec_bert(doc2vec_model, bert_model, tokenizer, items[4], jobs) # резюме дизайнера

[('В рекламное агентство требуется творческий копирайтер для создания текстов различной тематики. Обязанности: написание рекламных и PR-текстов, создание контента для сайтов и социальных сетей, работа над креативными концепциями. Требуется опыт написания текстов, креативность, отличное владение русским языком. Мы предлагаем возможность реализации творческого потенциала, работу над интересными проектами и достойную оплату труда.',
  67.82),
 ('Требуется инженер-конструктор в инженерно-проектное бюро. Задачи: разработка проектной документации, расчеты и моделирование в CAD-системах, сопровождение проектов на всех этапах. Требования: высшее техническое образование, опыт работы в проектировании, знание AutoCAD и других CAD-систем. Предлагаем интересные проекты, работу в команде профессионалов, стабильную заработную плату и возможности для карьерного роста.',
  61.43),
 ('ИТ-компания ищет талантливого программиста 1C для разработки и поддержки корпоративных приложений. Основные задачи: разр

# Summary

**Useful links:**
- [CV Job Matching using Doc2Vec](https://github.com/kirudang/CV-Job-matching/blob/main/Matching_Algorithm.ipynb)

**Datasets:**
- [Резюме из ЕЦП «Работа в России»](https://trudvsem.ru/opendata/datasets)
- [Вакансии всех регионов России из ЕЦП «Работа в России»](https://trudvsem.ru/opendata/datasets)

**Approaches:**
- Doc2Vec _(gensim)_
- BERT _(DeepPavlov/rubert-base-cased)_
- Doc2Vec + BERT

**Pipeline:**
1. Converting tabular data to text format
2. Data cleanup and tokenization
3. Creating a Doc2Vec embedding space
4. Creating functions that accept texts, create vectors in Doc2Vec, create embiddings in BERT, concatenate them and then compare based on cosine_similarity